In [1]:
# def load_data(loc:str, index:list):
#     df = pandas.read_csv(f'data/{loc}_solar19.csv')
#     df['idx'] = index
#     df = df.set_index('idx')
#     return df
# idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]
# weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
# weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
# weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
# weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]""
# demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
# demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
# demand_ny = DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga
# ng_price = make_henry_price_df(
#     file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
# ng_price = ng_price.drop(columns= 'scales')
# ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
# ng_price = pandas.concat([ng_price])

In [2]:
import sys

sys.path.append('src')

In [3]:
from energiapy.components import *
from energiapy.environ import *
from pandas import DataFrame
from dataclasses import fields, field, dataclass

In [4]:
s = Scenario(default=True)

In [5]:
# s.h2 = Horizon({'days': 2, 'hours': 12})
s.hor = Horizon(discretizations=[2, 12], nested=True)
s.scales
s.net = Network(locs = ['madgaon', 'ponje', 'cacoda'])  # , link_all=True)
s.nodes

TypeError: 'str' object cannot be interpreted as an integer

In [8]:
s.net.__dict__

{'nested': ['madgaon', 'ponje', 'cacoda'],
 'label': None,
 'locs': [1],
 'label_locs': None,
 'link_all': False,
 'name': 'net',
 '_named': True,
 '_model': Model|:s:|,
 '_tuplered': False,
 '_partition_list': [1],
 'name_partitions': ['node1']}

In [7]:
s.hor.__dict__ 

{'nested': True,
 'label': None,
 'discretizations': [1, 2, 12],
 'label_scales': None,
 'name': 'hor',
 '_named': True,
 '_model': Model|:s:|,
 '_tuplered': False,
 '_partition_list': [1, 2, 12],
 'name_partitions': ['t1', 't2', 't12']}

In [ ]:

s.alink = Linkage(source=s.madgaon, sink=s.ponje, bi=True, distance=50)
s.blink = Linkage(source=s.ponje, sink=s.cacoda, bi=False, distance=80)
s.clink = Linkage(source=s.cacoda, sink=s.madgaon, bi=True, distance=100)
s.dlink = Linkage(source=s.madgaon, sink=s.ponje, bi=False, distance=200)

In [7]:
a = DataFrame({'a': list(range(2))})
b = DataFrame({'b': list(range(24))})

In [8]:
s.csh = Cash(
    spend={
        s.madgaon: True,
        s.ponje: {s.t2: (2, a)},
        s.cacoda: a,
        s.network: {s.t2: [5, 7]},
    },
    # label='cash',
    basis='USD',
)

True
(2,    a
0  0
1  1)
   a
0  0
1  1
[5, 7]


c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:230: InconsistencyWarning: 
csh.spend:Inconsistent temporal scale for ponje at t2.
Updating to t1
Set Scneario.ok_inconsistent=False, to enforce strictly
  warn(


In [10]:
s.cash.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [11]:
s.H2 = Resource(
    # sell={s.t0: [0, 4], s.t1: [a, 34], s.t2: (a, b)},
    sell_price={
        s.madgaon: {s.t1: (2, a)},
        s.ponje: {s.t1: (2, b)},
        s.cacoda: 300,
    },
)

(2,    a
0  0
1  1)
(2,      b
0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
10  10
11  11
12  12
13  13
14  14
15  15
16  16
17  17
18  18
19  19
20  20
21  21
22  22
23  23)
300


c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:230: InconsistencyWarning: 
H2.sell_price:Inconsistent temporal scale for ponje at t1.
Updating to t2
Set Scneario.ok_inconsistent=False, to enforce strictly
  warn(


In [12]:
s.H2.parameters

[Price^sell[csh, H2, cacoda, t0],
 Price^sell[csh, H2, madgaon, t1],
 Price^sell[csh, H2, ponje, t2]]

In [13]:
s.H2.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [14]:
s.Solar = Resource(buy=True, basis='MW', label='Solar Power')


s.Wind = Resource(buy=a, basis='MW', label='Wind Power')


s.Power = Resource(basis='MW', label='Power generated')


s.Uranium = Resource(
    buy=DataFrame({'a': [i for i in range(24)]}),
    buy_price=42.70 / (250 / 2),
    basis='kg',
    label='Uranium',
)


# s.H2_L = Resource(sell=(0, 23), basis='tons', label='Hydrogen')


# s.CO2_AQoff = Resource(basis='tons', label='Carbon dioxide - sequestered')


s.H2O = Resource(buy=(20, 50), buy_price=b, basis='tons', label='Water')


s.CH4 = Resource(buy=[20, 40], buy_price=20, basis='tons', label='Natural gas')


s.CO2 = Resource(basis='tons', label='Carbon dioxide', block='Resource')


s.CO2_Vent = Resource(
    basis='tons',
    label='Carbon dioxide - Vented',
    sell_price=(2, 20),
    sell_emission={s.gwp: b},
)


s.O2 = Resource(sell=[20, True], basis='tons', label='Oxygen')


s.CO2_DAC = Resource(basis='tons', label='Carbon dioxide - captured')
s.Power = Resource(buy=[0, a], basis='MW', label='Power generated')

True
   a
0  0
1  1
     a
0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
10  10
11  11
12  12
13  13
14  14
15  15
16  16
17  17
18  18
19  19
20  20
21  21
22  22
23  23
0.3416
0.3416
0.3416
(20, 50)
     b
0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
10  10
11  11
12  12
13  13
14  14
15  15
16  16
17  17
18  18
19  19
20  20
21  21
22  22
23  23
     b
0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
10  10
11  11
12  12
13  13
14  14
15  15
16  16
17  17
18  18
19  19
20  20
21  21
22  22
23  23
     b
0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
10  10
11  11
12  12
13  13
14  14
15  15
16  16
17  17
18  18
19  19
20  20
21  21
22  22
23  23
[20, 40]
20
20
20
(2, 20)
(2, 20)
(2, 20)
     b
0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
10  10
11  11
12  12
13  13
14  14
15  15
16  16
17  17
18  18
19  19
20  20
21  21
22  22
23  23
     b
0    0
1    1
2    2
3    

c:\Users\cacodcar\energiapy\src\energiapy\environ\_scenario\_ok.py:54: OverWriteWarning: 
A Power was already defined.
Overwriting. Set Scenario.ok_overwrite=False, to enforce strictly
This should not cause any modeling issues.
Check Scenario defaults if unintended.
Only one instance of Horizon, Network, Cash, and Land are allowed
  warn(OverWriteWarning(cmp))


In [15]:
s.LiR = Material(
    use_emission={s.gwp: {s.t2: 1.484}},
    use={s.ponje: 1.5},
    basis='kg',
    label='Lithium Reserves',
    citation='Nelson Bunyui Manjong (2021)',
)

1.5
1.484
1.484
1.484


In [16]:
s.LiR.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [17]:
s.Power

Power

In [19]:
fields(Process)

(Field(name='basis',type=<class 'str'>,default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x00000185FED9E4E0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 Field(name='citation',type=<class 'dict'>,default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x00000185FED9E4E0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 Field(name='block',type=<class 'str'>,default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x00000185FED9E4E0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 Field(name='introduce',type=<class 'str'>,default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x00000185FED9E4E0>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD),
 Field(name='retire',type=<class 'str'>,default=None,default_factory=<datacl

In [20]:
Process.inputs()

['setup_time',
 'setup_use',
 'setup_emission',
 'capex',
 'opex',
 'produce',
 'capacity',
 'emit',
 'setup_emission',
 'use_emission',
 'buy_emission',
 'sell_emission',
 'loss_emission',
 'use',
 'use_cost',
 'buy',
 'sell',
 'buy_price',
 'sell_price',
 'credit',
 'penalty']

In [18]:
s.WF = Process(
    conversion={s.Power: {X(0): {s.Wind: -1}, X(1): {s.Wind: -1}}},
    produce=[(0, 3), (3, 5)],
    capex={s.madgaon: {s.t1: {1462000, I(202233)}}},
    opex={4953, I(70)},
    capacity=1000,
    setup_emission={s.gwp: 50, s.odp: 2900},
    setup_use={s.LiR: 1.5, s.land: 30},
    locations=[s.madgaon, s.ponje],
    label='Wind mill array',
    basis='MW',
)


s.PV = Process(
    sell_price={s.Power: 30},
    credit={s.Power: DataFrame({'a': [2]})},
    conversion={s.Power: {X(1): {s.Solar: -1.2}, X(2): {s.Solar: -1.5}}},
    capex={X(0): 1333, X(1): 1444, X(2): 1555},
    opex=22623,
    capacity={X(0): 3000, X(1): 4000, X(2): 5000},
    locations=s.cacoda,
    label='Solar PV',
    basis='MW',
)


s.SMRH = Process(
    setup_use={s.land: 50},
    conversion={
        s.H2: {
            s.Power: -1.11,
            s.CH4: -3.76,
            s.H2O: -23.7,
            s.CO2_Vent: 1.03,
            s.CO2: 9.332,
        }
    },
    capex={
        2520000,
    },
    opex={51.5, I(945000)},
    capacity=[1000],
    label='Steam methane reforming + CCUS',
)

s.NGCC = Process(
    buy_price={s.CH4: 4, s.H2O: 20},
    conversion={
        s.Power: {
            s.CH4: -0.108,
            s.H2O: -10,
            s.CO2_Vent: 0.297 * 0.05,
            s.CO2: 0.297 * 0.95,
        }
    },
    capex=2158928,
    opex={4090, I(53320)},
    capacity=[1, 100],
    label='NGCC + 95% CC',
)
s.SMR = Process(
    capex=2400,
    opex={0.03, I(800)},
    conversion={s.H2: {s.Power: -1.11, s.CH4: -3.76, s.H2O: -23.7, s.CO2_Vent: 9.4979}},
    capacity=1000,
    label='Steam methane reforming',
)
s.H2FC = Process(
    buy_price={s.H2: 2},
    sell={s.Power: 20},
    conversion={s.Power: {s.H2: -0.050}},
    capex=1.6 * 10**6,
    opex=3.5,
    capacity=[1000, 2000],
    label='hydrogen fuel cell',
)
s.DAC = Process(
    capex=0.02536,
    opex=0.634,
    conversion={s.CO2_DAC: {s.Power: -0.193, s.H2O: -4.048}},
    capacity=1000,
    label='Direct air capture',
)
# s.PSH = Process(conversion = {s.Power: 0.6}, capex = 3924781, fopex= 17820, vopex = 512.5, store = 10000, capacity=1000, label='Pumped storage hydropower', basis = 'MW')
s.ASMR = Process(
    buy={s.Uranium: 40},
    conversion={s.Power: {s.Uranium: -4.17 * 10 ** (-5), s.H2O: -3.364}},
    capex=7988951,
    opex=I(0.04 * 0.730),
    capacity=1000,
    label='Small modular reactors (SMRs)',
    locations=s.madgaon,
)
s.AWE = Process(
    setup_use={s.land: (0, 20)},
    conversion={s.H2: {s.Power: -1, s.O2: 0.7632, s.H2O: -0.1753}},
    capex={1.1 * 10**6, I(20)},
    opex=I(16918),
    capacity=True,
    label='Alkaline water electrolysis (AWE)',
    citation='Demirhan et al. 2018 AIChE paper',
)

1.5
1.5
1.5
30
30
30
50
50
50
2900
2900
2900
{1462000, i}
{4953, i}
{4953, i}
{4953, i}
[(0, 3), (3, 5)]
1000


InputTypeError: setup_emission{(gwp, WF, cacoda, t0): 50[gwp, WF, cacoda, t0], (gwp, WF, madgaon, t0): 50[gwp, WF, madgaon, t0], (gwp, WF, ponje, t0): 50[gwp, WF, ponje, t0], (odp, WF, cacoda, t0): 2900[odp, WF, cacoda, t0], (odp, WF, madgaon, t0): 2900[odp, WF, madgaon, t0], (odp, WF, ponje, t0): 2900[odp, WF, ponje, t0]} [<class 'energiapy.environ.datum.Datum'>] is not a valid input for WF.setup_emission.
Exact attrs can only take certain types.
Check Scenario.attr.bounds() for Bound attrs.
Check Scenario.attr.exacts() for Exact attrs.
Checkout energiapy.core.aliases.is_input for valid input types.

In [34]:
getattr(s.AWE.taskmaster, 'capex').__dict__

AttributeError: 'Scenario' object has no attribute 'AWE'

In [29]:
s.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
s.taskmaster.exacts()

In [ ]:
Process().inputs()

In [ ]:
s.LiI = Storage(capacity=[2000], inventory=s.Power, capacity_in=[40], capacity_out=[20])
s.LiI2 = Storage(capacity=[2000], inventory={s.Power: 0.6})
s.H2Stg = Storage(
    capacity=[2000], inventory={s.H2: {X(0): {s.Power: 0.8}, X(1): {s.Power: 0.9}}}
)

In [ ]:
help(Process)

In [ ]:
s.pprint()

In [ ]:
print(len(s.variables))
print(len(set(s.variables)))
print(len(set([v.name for v in s.variables])))
print()
print(len(s.parameters))
print(len(set(s.parameters)))
print(len(set([v.name for v in s.parameters])))
print()
print(len(s.constraints))
print(len(set(s.constraints)))
print(len(set([v.name for v in s.constraints])))
print()
print(len(s.indices))
print(len(set(s.indices)))
print(len(set([v.name for v in s.indices])))

In [ ]:
Transit()._spatials()

In [ ]:
for block in s.program.blocks:
    print(block.attr_dispositions)

In [ ]:
is_(s.program.variables[5], s.program.variables[6])

In [ ]:
len(set(s.program.variables))

In [ ]:
s.program.variables[6]

In [ ]:
s.LiI.capacity

In [ ]:
getattr(s.LiI, 'capacity')

In [ ]:
s.attr.bounds()

In [ ]:
s.components

In [ ]:
s.train = Transit(
    freight=s.H2,
    ship={s.H2: [300]},
    capacity=[5, 400],
    setup_use={s.land: 20},
    setup_emission={s.odp: 0.1},
    capex={s.alink: 200, s.blink: 300, s.clink: 400},
    opex=100,
    label='Train',
    basis='passengers',
    linkages=[s.alink],
)

In [ ]:
s.pprint()